<a href="https://colab.research.google.com/github/funkytaco/AI-Vocal-Remover/blob/master/so_vits_svc_4_0_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Check GPU
!nvidia-smi

In [ ]:
#@title Setup 1 (just run this once).
import os
import glob
!git clone https://github.com/effusiveperiscope/so-vits-svc -b eff-4.0
os.chdir('so-vits-svc')
# install requirements one-at-a-time to ignore exceptions
!cat requirements.txt | xargs -n 1 pip install --extra-index-url https://download.pytorch.org/whl/cu116
!pip install praat-parselmouth
!pip install ipywidgets
!pip install huggingface_hub
!pip install pip==23.0.1 # fix pip version for fairseq install
!pip install fairseq==0.12.2
!pip uninstall -y matplotlib
!pip install --no-cache-dir matplotlib
!jupyter nbextension enable --py widgetsnbextension
existing_files = glob.glob('/content/**/*.*', recursive=True)

In [ ]:
#@title Setup 2 (just run this once)
import os
os.chdir('/content/so-vits-svc') # force working-directory to so-vits-svc - this line is just for safety and is probably not required

from zipfile import ZipFile
from pathlib import Path
import tarfile
import gdown
import urllib
import os
import subprocess
from tqdm import tqdm

class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)
        
class Downloader2:
    LINUX_MEGATOOLS_URL = ("https://megatools.megous.com/builds/builds/"
        "megatools-1.11.1.20230212-linux-x86_64.tar.gz")
    def __init__(self):
        self.mega_setup()
        pass

    def mega_setup(self):
        MEGATOOLS_TAR_PATH = Downloader2.LINUX_MEGATOOLS_URL.split("/")[-1]
        MEGATOOLS_FOLDER_PATH = (
            Downloader2.LINUX_MEGATOOLS_URL.split("/")[-1].removesuffix(
                ".tar.gz"))
        self.abs_megatools_path = os.path.abspath(MEGATOOLS_FOLDER_PATH)

        urllib.request.urlretrieve(
            url=Downloader2.LINUX_MEGATOOLS_URL,
            filename=MEGATOOLS_TAR_PATH)
        with tarfile.open(MEGATOOLS_TAR_PATH) as tar:
            tar.extractall()
            tar.close()
        os.unlink(MEGATOOLS_TAR_PATH)
        assert os.path.exists(self.abs_megatools_path)

        pass

    def megadown(self, url, filename):
        cmd = (os.path.join(self.abs_megatools_path,'megatools')+
            " dl "+"--print-names "+(
            "--path "+filename+" " if filename else "")+url)
        proc = subprocess.run(cmd, shell=True)
        if proc.returncode != 0:
            raise Exception('megadown failed -- cmd: '+cmd)
        return filename

    def request_url_with_progress_bar(self, url, filename):
        def download_url(url, filename):
            with DownloadProgressBar(unit='B', unit_scale=True,
                miniters=1, desc=url.split('/')[-1]) as t:
                return urllib.request.urlretrieve(
                    url, filename=filename, reporthook=t.update_to)
        return download_url(url, filename)

    def download(self, url, filename):
        if "drive.google.com" in url:
            print("Downloading Google Drive file "+url)
            return gdown.download(url, filename, quiet=False, fuzzy=True)
        elif "mega.nz" in url:
            print("Downloading MEGA file "+url)
            # There is no other way to determine the file name
            # from megatools prior to downloading without authentication
            # so we set it to a placeholder
            import uuid
            return self.megadown(url, filename=str(uuid.uuid4())+".zip")
        else:
            print("Downloading direct "+url)
            local_filename, headers = (
                self.request_url_with_progress_bar(url, filename))
            return local_filename


import fnmatch
def default_next(x):
    try:
        return next(x)
    except StopIteration:
        return None
    
def zip_extract(zipfile, model_dir):
    model_folder_name = Path(zipfile).stem
    model_folder_path = os.path.join(model_dir,
        Path(zipfile).stem)
    with ZipFile(zipfile, 'r') as f:
        member_infos = f.infolist()
        generator = default_next(
            x for x in member_infos if fnmatch.fnmatch(x.filename, '*G_*.pth'))
        config_json = default_next(
            x for x in member_infos if fnmatch.fnmatch(x.filename, '*.json'))
        cluster_pt = default_next(
            x for x in member_infos if fnmatch.fnmatch(x.filename, '*.pt'))

        generator.filename = generator.filename.split('/')[-1]
        config_json.filename = config_json.filename.split('/')[-1]

        if (not generator):
            print("Could not find G_*.pth in "+zipfile)
            return
        if (not config_json):
            print("Could not find config.json in "+zipfile)
            return
        f.extract(generator, path=model_folder_path)
        f.extract(config_json, path=model_folder_path)

        if cluster_pt:
            cluster_pt.filename = cluster_pt.filename.split('/')[-1]
            f.extract(cluster_pt, path=model_folder_path)
    print("Cleaning "+zipfile)
    os.remove(zipfile)

import huggingface_hub
import os
import shutil

class HFModels:
    def __init__(self, repo = "therealvul/so-vits-svc-4.0", 
            model_dir = "hf_vul_models"):
        self.model_repo = huggingface_hub.Repository(local_dir=model_dir,
            clone_from=repo, skip_lfs_files=True)
        self.repo = repo
        self.model_dir = model_dir

        self.model_folders = sorted(os.listdir(model_dir))
        print(self.model_folders)
        self.model_folders.remove('.git')
        self.model_folders.remove('.gitattributes')

    def list_models(self):
        return self.model_folders

    # Downloads model;
    # copies config to target_dir and moves model to target_dir
    def download_model(self, model_name, target_dir):
        if not model_name in self.model_folders:
            raise Exception(model_name + " not found")
        model_dir = self.model_dir
        charpath = os.path.join(model_dir,model_name)

        gen_pt = next(x for x in os.listdir(charpath) if x.startswith("G_"))
        cfg = next(x for x in os.listdir(charpath) if x.endswith("json"))
        try:
          clust = next(x for x in os.listdir(charpath) if x.endswith("pt"))
        except StopIteration as e:
          print("Note - no cluster model for "+model_name)
          clust = None

        if not os.path.exists(target_dir):
            os.makedirs(target_dir, exist_ok=True)

        gen_dir = huggingface_hub.hf_hub_download(repo_id = self.repo,
            filename = model_name + "/" + gen_pt) # this is a symlink
        
        if clust is not None:
          clust_dir = huggingface_hub.hf_hub_download(repo_id = self.repo,
              filename = model_name + "/" + clust) # this is a symlink
          shutil.move(os.path.realpath(clust_dir), os.path.join(target_dir, clust))
          clust_out = os.path.join(target_dir, clust)
        else:
          clust_out = None

        shutil.copy(os.path.join(charpath,cfg),os.path.join(target_dir, cfg))
        shutil.move(os.path.realpath(gen_dir), os.path.join(target_dir, gen_pt))

        return {"config_path": os.path.join(target_dir,cfg),
            "generator_path": os.path.join(target_dir,gen_pt),
            "cluster_path": clust_out}

# Example usage
# vul_models = HFModels()
# print(vul_models.list_models())
# print("Applejack (singing)" in vul_models.list_models())
# vul_models.download_model("Applejack (singing)","models/Applejack (singing)")
downloader = Downloader2()
print("Finished!")

In [ ]:
#@title Download ContentVec (just run this once)
os.chdir('/content/so-vits-svc') # force working-directory to so-vits-svc - this line is just for safety and is probably not required
downloader.download("https://huggingface.co/therealvul/so-vits-svc-4.0-init/resolve/main/checkpoint_best_legacy_500.pt", filename="hubert/checkpoint_best_legacy_500.pt")

In [ ]:
#@title Setup HF Downloads (run cell and click buttons to download models)
from ipywidgets import widgets, Layout
hf_repos = [HFModels(), HFModels("OlivineEllva/so-vits-svc-4.0-models",
    "repositories/hf_oe_model")]

import functools

for hf_repo in hf_repos:
  models = hf_repo.list_models()
  for model in models:
    btn = widgets.Button(description=model, layout=Layout(
        width='50%', height="30px"))
    def button_eventhandler(but, repo):
      repo.download_model(but.description, "models/"+but.description)
    btn.on_click(functools.partial(button_eventhandler, repo=hf_repo))
    display(btn)

In [ ]:
#@title Other Downloads (.zip) Step o.1
#@markdown Please note that 3.0 models are incompatible with 4.0.
#@markdown After running this step, be sure to run the below step (o.2).

#@markdown Supported URL types: 
#@markdown * Google Drive zip
#@markdown * MEGA zip
#@markdown * Direct zip (+HuggingFace /resolve/ link)

import re
model_url = "https://huggingface.co/Amo/so-vits-svc-4.0_GA/resolve/main/ModelsZip/TreeHugger_69k.zip" #@param {"type": "string"}
if "huggingface.co" in model_url.lower():
  model_zip_path = downloader.download(re.sub(r"/blob/","/resolve/",model_url), 
           filename=os.path.join(os.getcwd(),model_url.split("/")[-1]))
else:
  model_zip_path = downloader.download(model_url, filename=None)

In [ ]:
#@title Extract .zip Downloads - Step o.2
# download speaker model files into 'models' directory
import glob, os, shutil
from pathlib import Path
os.makedirs('models', exist_ok=True)

zip_extract(model_zip_path, 'models')

**!!! Please restart the runtime before running the next cell !!!**

Note: If you need to download more models you may need to re-run setup 2 after restarting the runtime

In [ ]:
#@title Open the file explorer on the left of your screen and drag-and-drop an audio file into so-vits-svc/raw. Then run the below cell.
#@markdown As of 4/21/23 this cell works differently! Please re-read the above instruction!

#@markdown If you get an error relating to numpy please restart the runtime. (Runtime > Restart runtime)

import os
import glob
import json
import copy
import logging
import io
from ipywidgets import widgets, Layout
from pathlib import Path
from IPython.display import Audio, display

os.chdir('/content/so-vits-svc')

import torch
from inference import infer_tool
from inference import slicer
from inference.infer_tool import Svc
import soundfile
import numpy as np

MODELS_DIR = "models"
OUTPUTS_DIR = "results"
INPUTS_DIR = "raw"

import base64
import hashlib
from typing import Callable

import ipywidgets
from IPython.display import HTML, display

class DownloadButton(ipywidgets.Button):
    """Download button with dynamic content

    The content is generated using a callback when the button is clicked.
    """

    def __init__(self, filename: str, contents: Callable[[], str], **kwargs):
        super(DownloadButton, self).__init__(**kwargs)
        self.filename = filename
        self.contents = contents
        self.on_click(self.__on_click)

    def __on_click(self, b):
        contents: bytes = self.contents()
        b64 = base64.b64encode(contents)
        payload = b64.decode()
        digest = hashlib.md5(contents).hexdigest()  # bypass browser cache
        id = f'dl_{digest}'

        display(HTML(f"""
<html>
<body>
<a id="{id}" download="{self.filename}" href="data:text/csv;base64,{payload}" download>
</a>

<script>
(function download() {{
document.getElementById('{id}').click();
}})()
</script>

</body>
</html>
"""))

def get_speakers():
  speakers = []
  for _,dirs,_ in os.walk(MODELS_DIR):
    for folder in dirs:
      cur_speaker = {}
      # Look for G_****.pth
      g = glob.glob(os.path.join(MODELS_DIR,folder,'G_*.pth'))
      if not len(g):
        print("Skipping "+folder+", no G_*.pth")
        continue
      cur_speaker["model_path"] = g[0]
      cur_speaker["model_folder"] = folder

      # Look for *.pt (clustering model)
      clst = glob.glob(os.path.join(MODELS_DIR,folder,'*.pt'))
      if not len(clst):
        print("Note: No clustering model found for "+folder)
        cur_speaker["cluster_path"] = ""
      else:
        cur_speaker["cluster_path"] = clst[0]

      # Look for config.json
      cfg = glob.glob(os.path.join(MODELS_DIR,folder,'*.json'))
      if not len(cfg):
        print("Skipping "+folder+", no config json")
        continue
      cur_speaker["cfg_path"] = cfg[0]
      with open(cur_speaker["cfg_path"]) as f:
        try:
          cfg_json = json.loads(f.read())
        except Exception as e:
          print("Malformed config json in "+folder)
        for name, i in cfg_json["spk"].items():
          cur_speaker["name"] = name
          cur_speaker["id"] = i
          if not name.startswith('.'):
            speakers.append(copy.copy(cur_speaker))

    return sorted(speakers, key=lambda x:x["name"].lower())

logging.getLogger('numba').setLevel(logging.WARNING)
chunks_dict = infer_tool.read_temp("inference/chunks_temp.json")
slice_db = -40
wav_format = 'wav'

from functools import partial
import zipfile
import io

class InferenceGui():
  def __init__(self):
    self.speakers = get_speakers()
    self.speaker_list = [x["name"] for x in self.speakers]
    self.speaker_box = widgets.Dropdown(
        options = self.speaker_list
    )
    display(self.speaker_box)

    os.makedirs(os.path.join('/content/so-vits-svc/',OUTPUTS_DIR),
        exist_ok=True)

    def convert_cb(btn):
      self.convert()
    def clean_cb(btn, dir):
      self.clean(dir)
    def dl_zip_contents():
      input_filepaths = [f for f in glob.glob(
         os.path.join('/content/so-vits-svc/',OUTPUTS_DIR)+'/*.*'
         , recursive=True)
      if any(f.endswith(ex) for ex in ['.wav','.flac','.mp3','.ogg','.opus'])]
      memory_zip = io.BytesIO()
      with zipfile.ZipFile(memory_zip, mode="w") as z:
        for f in input_filepaths:
          z.write(f, arcname=f.split('/')[-1])
      return memory_zip.getvalue()

    self.convert_btn = widgets.Button(description="Convert", layout=Layout(
        width='30%', height="30px"))
    self.convert_btn.on_click(convert_cb)
    self.clean_input_btn = widgets.Button(description=
        "Delete all files in input folder", layout=Layout(
        width='30%', height="30px"))
    self.clean_input_btn.on_click(partial(clean_cb, dir=INPUTS_DIR))
    self.clean_output_btn = widgets.Button(description=
        "Delete all files in output folder", layout=Layout(
        width='30%', height="30px"))
    self.clean_output_btn.on_click(partial(clean_cb, dir=OUTPUTS_DIR))
    self.dl_output_btn = DownloadButton(description=
        "Download all files in output folder as .zip",
        filename="dl.zip", contents = dl_zip_contents, layout=Layout(
        width='30%', height="30px"))

    self.trans_tx = widgets.IntText(value=12, description='Transpose')
    self.cluster_ratio_tx = widgets.FloatText(value=0.0, 
      description='Clustering Ratio')
    self.noise_scale_tx = widgets.FloatText(value=0.4, 
      description='Noise Scale')
    self.auto_pitch_ck = widgets.Checkbox(value=False, description=
      'Auto pitch f0 (do not use for singing)')

    display(self.trans_tx)
    display(self.cluster_ratio_tx)
    display(self.noise_scale_tx)
    display(self.auto_pitch_ck)
    display(self.convert_btn)
    display(self.clean_input_btn)
    display(self.clean_output_btn)
    display(self.dl_output_btn)

  def convert(self):
    trans = int(self.trans_tx.value)
    speaker = next(x for x in self.speakers if x["name"] == 
          self.speaker_box.value)
    spkpth2 = os.path.join(os.getcwd(),speaker["model_path"])
    print(spkpth2)
    print(os.path.exists(spkpth2))

    svc_model = Svc(speaker["model_path"], speaker["cfg_path"], 
      cluster_model_path=speaker["cluster_path"])
    
    input_filepaths = [f for f in glob.glob(
        os.path.join('/content/so-vits-svc/',
        INPUTS_DIR)+'/*.*', recursive=True)
     if any(f.endswith(ex) for ex in ['.wav','.flac','.mp3','.ogg','.opus'])]
    for name in input_filepaths:
      print("Converting "+os.path.split(name)[-1])
      infer_tool.format_wav(name)

      wav_path = str(Path(name).with_suffix('.wav'))
      wav_name = Path(name).stem
      chunks = slicer.cut(wav_path, db_thresh=slice_db)
      audio_data, audio_sr = slicer.chunks2audio(wav_path, chunks)

      audio = []
      for (slice_tag, data) in audio_data:
          print(f'#=====segment start, '
              f'{round(len(data)/audio_sr, 3)}s======')
          
          length = int(np.ceil(len(data) / audio_sr *
              svc_model.target_sample))
          
          if slice_tag:
              print('jump empty segment')
              _audio = np.zeros(length)
          else:
              # Padding "fix" for noise
              pad_len = int(audio_sr * 0.5)
              data = np.concatenate([np.zeros([pad_len]),
                  data, np.zeros([pad_len])])
              raw_path = io.BytesIO()
              soundfile.write(raw_path, data, audio_sr, format="wav")
              raw_path.seek(0)
              _cluster_ratio = 0.0
              if speaker["cluster_path"] != "":
                _cluster_ratio = float(self.cluster_ratio_tx.value)
              out_audio, out_sr = svc_model.infer(
                  speaker["name"], trans, raw_path,
                  cluster_infer_ratio = _cluster_ratio,
                  auto_predict_f0 = bool(self.auto_pitch_ck.value),
                  noice_scale = float(self.noise_scale_tx.value))
              _audio = out_audio.cpu().numpy()
              pad_len = int(svc_model.target_sample * 0.5)
              _audio = _audio[pad_len:-pad_len]
          audio.extend(list(infer_tool.pad_array(_audio, length)))
          
      res_path = os.path.join('/content/so-vits-svc',OUTPUTS_DIR,
          f'{wav_name}_{trans}_key_'
          f'{speaker["name"]}.{wav_format}')
      soundfile.write(res_path, audio, svc_model.target_sample,
          format=wav_format)
      display(Audio(res_path, autoplay=True)) # display audio file
    pass

  def clean(self, folder_name):
     input_filepaths = [f for f in glob.glob(
         os.path.join('/content/so-vits-svc/',folder_name)+'/*.*'
         , recursive=True)
     if any(f.endswith(ex) for ex in ['.wav','.flac','.mp3','.ogg','.opus'])]
     for f in input_filepaths:
       os.remove(f)

inference_gui = InferenceGui()

Changelog

*   3/31/23: possible fix for mega zip downloads
*   4/14/23: possible more robust downloads
*   4/20/23: WIDER HF downloads buttons, "fix" unusual GDrive links
*   4/21/23: Interface changes
*   4/22/23: I forgot to add the layout import to the bottom cell whoops lol
